# **Static, partial equilibrium model**
### **Production module, example 4:**

*Example 1 sets up a nesting tree consisting of one tree (input-type), initializes the production module in the partial equilibrium framework, run (with some default initial values/parameter values), and calibrate the model.*

Import standard collection of packages:

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

## **Nesting structure:**

*Nesting trees:*

In [2]:
tree_in = {'Y': ['KY1','LY2'],
           'KY1': ['K','Y1'],
           'LY2': ['L','Y2']}
# tree_out = {'Y': ['Y1_out','Y2_out']}

*Initialize empty class with some name:*

In [3]:
nt = nesting_tree.nesting_tree(name='test')

*Add trees with various settings (NB: Using'normalized' versions throughout the entire tree can make the system of equations quite difficult to solve):*

*Input types:*

In [4]:
nt.add_tree(tree_in,'s1_in',**{'tree_name': 's1_in','version': 'std'}) # regular CES version.
# nt.add_tree(tree_in,'in',**{'tree_name': 'test_in','type_f': 'norm_CES'}) # normalized CES-version.
# nt.add_tree(tree_in,'in',**{'tree_name': 'test_in','type_f': 'MNL'}) # MNL input-like system.
# nt.add_tree(tree_in,'in',**{'tree_name': 'test_in','type_f': 'MNL', 'version': 'MNL_w_mu'}) # MNL input-like system with mu-parameters in the share functions.

*Output types:*

In [5]:
# nt.add_tree(tree_out,'s1_out',**{'tree_name': 's1_out','type_io': 'output','version': 'std'}) # regular CET version.
# nt.add_tree(tree_out,'out',**{'tree_name': 'test_out','type_io': 'output', 'type_f': 'norm_CET'}) # normalized CET version.
# nt.add_tree(tree_out,'out',**{'tree_name': 'test_out','type_io': 'output','type_f': 'MNL_out'})

*Retrieve sets/subsets/mappings from trees:*

In [6]:
nt.run_all()

## **Create production module:**

*Partial equilibrium model, production module from nesting tree:*

In [7]:
pm = PE.GPM_STA_PE.production(nt)

NameError: name 'PE' is not defined

*Write gams files to a repo called PE_P/Example_1, and export the 'settings' as well:*

In [ ]:
pm.df_write(repo=os.getcwd()+'\\PE_P\\Example_4',export_settings=True)

*Create model instance, checkpoint, and run:*

In [ ]:
pm.calibrate_model(repo=work_folder)

In [ ]:
pm.create_model_instance(repo=work_folder,name='baseline')
baseline = pm.model_instances['baseline'].ws.add_checkpoint()
pm.df_run(name='baseline',options_run={'checkpoint': baseline})

In baseline, unfix $\mu$ parameters, fix prices and quantities for input/output types:

In [ ]:
pm.model.settings.name

In [ ]:
job = pm.model_instances['baseline'].ws.add_job_from_string("""
qS.fx[n]$(out[n]) = qS.l[n];
qD.fx[n]$(inp[n]) = qD.l[n];
mu.lo[n,nn]$(map_all[n,nn] and inp[n]) = -inf;
mu.up[n,nn]$(map_all[n,nn] and inp[n]) = inf;
solve test using CNS;
""",**{'checkpoint': baseline})

In [ ]:
pm.model_instances['baseline'].__dict__

In [ ]:
job.run()

## **Calibration to national accounts-like data:**